# Anomaly detection made easy
This notebook contains code used to build material and visualizations used for the blog post anomaly detection made easy on medium.

In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

In [2]:
pd.options.plotting.backend = 'plotly'

## Setting up variables and seed

In [33]:
check_dates = [datetime(2022, 3, 11) + timedelta(days=x) for x in range(15)]

np.random.seed(42)

black = '#363833'
orange = '#c86a3e'
pink = '#d38bba'
lgreen = '#c8d7ab'
dgreen = '#4d5e38'
white = '#f0ece1'

## Plotting helper functions

In [37]:
def clean_fig(fig):
    fig.update_layout(
        plot_bgcolor='#FFF', title={'x': .5}, font_size=15,
        font_family='Arial', title_font_family="Times New Roman",
        font={'color': black}
    )
    fig.update_coloraxes(showscale=False)
    return fig

def write_image(fig, image_name, width=1200, height=600):
    image_path = f'images/{image_name}.png'
    fig.write_image(image_path, width=width, height=height)

## Estimation functions

In [4]:
def ema(values, span=20):
    alpha = 2 / (span + 1)
    result = []
    for i, v in enumerate(values):
        if i == 0:
            result.append(v)
        else:
            result.append(v * alpha + result[-1] * (1 - alpha))
    return result

def ema(values):
    alpha = 2 / (len(values) + 1)
    result = []
    for i, v in enumerate(values):
        if i == 0:
            result.append(v)
        else:
            result.append(v * alpha + result[-1] * (1 - alpha))
    return result[-1]

def ema_list(values):
    alpha = 2 / (len(values) + 1)
    result = []
    for i, v in enumerate(values):
        if i == 0:
            result.append(v)
        else:
            result.append(v * alpha + result[-1] * (1 - alpha))
    return result

def sma(values):
    return sum(values) / len(values)

def calc_beta(values):
    X = np.arange(len(values))

    X_ext = np.c_[np.ones(len(X)), X]
    X_gram = X_ext.T @ X_ext
    X_inv = np.linalg.inv(X_gram)
    Xy = X_ext.T @ values
    beta = X_inv @ Xy
    return beta

def ols(values):
    beta = calc_beta(values)
    # Multiplying by the length of values, sice we calculate
    # beta using np.arange(len(values)), meaning we multiply beta[1] with
    # The highest possible X value, thus giving us the prediction
    return beta[0] + beta[1] * len(values)

def std(values):
    mu = sum(values) / len(values)
    return math.sqrt(sum([(x - mu)**2 for x in values]) / len(values))

def calc_bounds(values, estimator_function, std_multiplier=2):
    e = estimator_function(values)
    s = std(values)
    return e, e - s * std_multiplier, e + s * std_multiplier

## What is an anomaly?

In [38]:
vals = [np.random.normal(1000, 50) for _ in range(10)]
idx = 6
vals[idx] += 80
upper = [np.mean(vals) + 100 * np.random.normal(1, .1) for _ in vals]
lower = [np.mean(vals) - 100 * np.random.normal(1, .1) for _ in vals]
x = np.arange(10)
fig = go.Figure([
    go.Scatter(x=x, y=upper, marker={'color': black}, mode='lines'),
    go.Scatter(x=x, y=lower, marker={'color': black}, fill='tonexty', mode='lines', fillcolor='rgba(240,236,225,20)'),
    go.Scatter(x=x, y=vals, mode='lines+markers', line={'width': 3, 'color': dgreen}, marker={'size': 10}),
    go.Scatter(
        x=[idx], y=[vals[idx]], mode='markers+text', marker={'size': 20, 'color': orange}
    )
])
fig.add_annotation(
    x=idx, y=vals[idx] + 30, text='<b>Anomaly!</b>', showarrow=False, arrowhead=2, arrowsize=2, arrowcolor='black', 
    font={'family': 'Times New Roman', 'size': 20, 'color': orange}
)
fig.update_layout(title='<b>Illustration of an anomaly</b>', showlegend=False)
fig = clean_fig(fig)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
write_image(fig, 'what_is_anomaly')
fig.show()

## Function to generate mock data
Sampled from a normal distribution, with alterations on separate day to create a fictional trend.

In [7]:
def generate_df(day_map):
    df = pd.DataFrame([
        [datetime(2022, 1, 1) + timedelta(days=x), np.random.normal(loc=6000, scale=500) + x*20]
        for x in range(100)
    ], columns=['date', 'nbr_sessions'])
    df['weekday'] = df.date.dt.day_name()
    for day, val in day_map.items():
        df.loc[df.weekday == day, 'nbr_sessions'] = df.loc[df.weekday == day, 'nbr_sessions'] * val
    df['nbr_sessions'] = df.nbr_sessions.astype(int)
    df['mean_seconds'] = np.random.normal(180, 10, len(df))
    df['total_seconds'] = df.nbr_sessions * df.mean_seconds
    return df

df = generate_df({'Wednesday': .9, 'Thursday': .8, 'Saturday': 1.1, 'Sunday': 1.2})
df.head(3)

,date,nbr_sessions,weekday,mean_seconds,total_seconds
0,2022-01-01,6269,Saturday,164.493366,1.031209e+06
1,2022-01-02,8335,Sunday,180.685630,1.506015e+06
2,2022-01-03,6033,Monday,169.376963,1.021851e+06


## Taking a look at our generated number of sessions over time

In [8]:
sdf = df[-50:].copy()
beta = calc_beta(sdf.nbr_sessions)
fig = go.Figure([
    go.Scatter(x=sdf.date, y=sdf.nbr_sessions, line={'color': dgreen}, name='Nbr Sessions'),
    go.Scatter(
        x=sdf.date, y=[beta[0] + beta[1] * x for x in range(len(sdf))], name='Trend Line',
        line={'color': orange, 'dash': 'dash'}
    )
])
fig.update_yaxes(range=[0, sdf.nbr_sessions.max() + 100])
fig.update_layout(title='<b>Number of sessions per day</b>')
fig = clean_fig(fig)
gridcolor = 'rgba(165,160,155,10)'
fig.update_xaxes(showline=True, linewidth=2, linecolor=black, gridcolor=gridcolor)
fig.update_yaxes(showline=True, linewidth=2, linecolor=black, gridcolor=gridcolor)
write_image(fig, 'nbr_sessions_per_day')
fig.show()

In [9]:
fig = px.box(
    df, x='weekday', y='nbr_sessions', 
    category_orders={'weekday': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']},
    color_discrete_sequence=[dgreen], title='<b>Box plot of the number of session distributions per day</b>'
)
fig = clean_fig(fig)
fig.update_yaxes(showline=False, linewidth=2, linecolor=black, gridcolor=gridcolor)
write_image(fig, 'box_plot_sessions')
fig.show()

# Building a predictor for our expected value
Gonna take three approaches:
* Simple Moving Average (SMA)
* Exponential Moving Average (EMA)
* Ordinary Least Squares (OLS)

In [41]:
r = 15
br = r - 3
sample_df = df[-r:].sort_values('date').copy()
beta = calc_beta(sample_df.nbr_sessions[:br])
bline = [beta[0] + beta[1] * v for v in np.arange(br)]
fig = go.Figure([
    go.Scatter(
        x=np.arange(r), y=sample_df.nbr_sessions, mode='lines+markers', 
        line={'color': dgreen, 'width': 3}, name='True Values'
    ),
    go.Scatter(x=np.arange(br), y=bline, mode='lines', line={'color': orange, 'width': 3}, name='OLS Line'),
    go.Scatter(x=[br - 1], y=[bline[-1]], marker={'color': orange, 'size': 12}, showlegend=False)
])
fig.update_layout(title='<b>Example of how estimation using OLS works</b>')
fig = clean_fig(fig)
fig.update_xaxes(showticklabels=True, showline=True, linewidth=2, linecolor=black, gridcolor=gridcolor)
fig.update_yaxes(showticklabels=False, showline=True, linewidth=2, linecolor=black, gridcolor=gridcolor)
fig.add_annotation(
    x=br - 1, y=bline[-1] + 50, text='Expected Value',
    ax=10, ay=-50,
    showarrow=True, arrowhead=2, arrowsize=2, arrowcolor=black, 
    font={'family': '<b>Times New Roman</b>', 'size': 20, 'color': orange}
)
write_image(fig, 'example_ols')
fig.show()

/var/folders/7v/vhfr1rrs3416c37q5fj21g1m0000gn/T/ipykernel_45664/2951168898.py:4: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.



In [11]:
dates, orig, sma_l, ema_l, ols_l = [], [], [], [], []

for d in check_dates:
    subset_df = df[df.date <= d].tail(15).copy()
    values = subset_df.nbr_sessions.values
    dates.append(d)
    orig.append(values[-1])
    sma_l.append(sma(values))
    ema_l.append(ema(values))
    ols_l.append(ols(values))

fig = go.Figure([
    go.Scatter(x=dates, y=sma_l, name='SMA', mode='lines', line={'dash': 'dash', 'width': 3, 'color': pink},),
    go.Scatter(x=dates, y=ema_l, name='EMA', mode='lines', line={'dash': 'dash', 'width': 3, 'color': dgreen}),
    go.Scatter(x=dates, y=ols_l, name='OLS', mode='lines', line={'dash': 'dash', 'width': 3, 'color': lgreen}),
    go.Scatter(x=dates, y=orig, name='Nbr Sessions', mode='lines', line={'width': 3, 'color': black}),
])
fig.update_layout(title='<b>Comparison of different estimation algorithms against true values</b>')
fig = clean_fig(fig)
write_image(fig, 'estimation_comparison')
fig.show()

In [42]:
anomaly_df = df.copy()
anomaly_date = '2022-03-17'
anomaly_df.loc[anomaly_df.date == anomaly_date, 'nbr_sessions'] -= 1500

dates, orig, lower, upper = [], [], [], []

for d in check_dates:
    subset_df = anomaly_df[anomaly_df.date <= d].tail(15).copy()
    values = subset_df.nbr_sessions.values
    dates.append(subset_df.date.tolist()[-1])
    orig.append(values[-1])
    e = ema(values)
    s = std(values)
    lower.append(e - s * 2)
    upper.append(e + s * 2)

fig = go.Figure([
    go.Scatter(x=dates, y=lower, mode='lines', marker={'color': black}, name='upper', showlegend=False),
    go.Scatter(
        x=dates, y=upper, mode='lines', marker={'color': black}, name='upper', 
        fill='tonexty', fillcolor='rgba(240,236,225,20)', showlegend=False
    ),
    go.Scatter(
        x=dates, y=orig, name='Nbr Sessions', line={'width': 4, 'color': dgreen},
        text=df[df.date.isin(dates)].weekday
        ),
])
fig.update_layout(title='<b>Boundaries built using EMA, anomaly found March 17</b>')
fig = clean_fig(fig)
fig.add_annotation(
    x=anomaly_date, y=anomaly_df.loc[anomaly_df.date == anomaly_date, 'nbr_sessions'].values[0],
    text='<b>Anomaly!</b>', ax=-100, ay=-20,
    showarrow=True, arrowhead=2, arrowsize=2, arrowcolor=black, 
    font={'family': 'Times New Roman', 'size': 20, 'color': orange}
)
write_image(fig, 'ema_boundaries')
fig.show()

In [43]:
anomaly_df = df.copy()
anomaly_df.loc[anomaly_df.date == '2022-03-17', 'nbr_sessions'] -= 1500

dates, orig, lower, upper = [], [], [], []

for d in check_dates:
    subset_df = anomaly_df.loc[(anomaly_df.date <= d) & (anomaly_df.weekday == d.strftime('%A'))].tail(15).copy()
    values = subset_df.nbr_sessions.values
    dates.append(subset_df.date.tolist()[-1])
    orig.append(values[-1])
    e = ema(values)
    s = std(values)
    lower.append(e - s * 2)
    upper.append(e + s * 2)

fig = go.Figure([
    go.Scatter(x=dates, y=lower, mode='lines', marker={'color': black}, name='upper', showlegend=False),
    go.Scatter(
        x=dates, y=upper, mode='lines', marker={'color': black}, name='upper', 
        fill='tonexty', fillcolor='rgba(240,236,225,20)', showlegend=False
    ),
    go.Scatter(
        x=dates, y=orig, name='Nbr Sessions', line={'width': 4, 'color': dgreen},
        text=df[df.date.isin(dates)].weekday
        ),
])
anomaly_date = '2022-03-18'
fig.add_annotation(
    x=anomaly_date, y=anomaly_df.loc[anomaly_df.date == anomaly_date, 'nbr_sessions'].values[0], 
    text='<b>Anomaly!</b>', ax=-100, ay=-20,
    showarrow=True, arrowhead=2, arrowsize=2, arrowcolor=black, 
    font={'family': 'Times New Roman', 'size': 20, 'color': orange}
)
fig.update_layout(title='<b>Boundaries built using EMA via looking at the same weekdays retrospectively</b>')
fig = clean_fig(fig)
write_image(fig, 'ema_boundaries_weekdays')
fig.show()

In [44]:
anomaly_df = df[df.date.isin(check_dates)].copy()
anomaly_df.loc[anomaly_df.date == '2022-03-17', 'nbr_sessions'] -= 1500
anomaly_df.loc[anomaly_df.date == '2022-03-17', 'total_seconds'] -= 1500 * 180
anomaly_df.loc[:, 'total_minutes'] = anomaly_df.total_seconds / 60

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=anomaly_df.date, y=anomaly_df.nbr_sessions, mode='lines', 
    line={'color': dgreen, 'width': 3}, name='Nbr Sessions'
    ), secondary_y=False)
fig.add_trace(go.Scatter(
    x=anomaly_df.date, y=anomaly_df.total_minutes, mode='lines', 
    line={'color': orange, 'width': 3}, name='Total Minutes'
    ), secondary_y=True)
fig.update_layout(title='<b>Comparison of nbr sessions and time spent trends</b>')
fig.update_yaxes(showticklabels=True)
fig = clean_fig(fig)
write_image(fig, 'comparison_session_total_time')
fig.show()

In [47]:
anomaly_df = df[df.date.isin(check_dates)].copy()
anomaly_df.loc[anomaly_df.date == '2022-03-17', 'nbr_sessions'] -= 1500

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=anomaly_df.date, y=anomaly_df.nbr_sessions, mode='lines', 
    line={'color': dgreen, 'width': 3}, name='Nbr Sessions'
    ), secondary_y=False)
fig.add_trace(go.Scatter(
    x=anomaly_df.date, y=anomaly_df.mean_seconds, mode='lines', 
    line={'color': orange, 'width': 3}, name='Mean Seconds'
    ), secondary_y=True)
fig.update_layout(title='<b>Comparison of nbr sessions and mean time per session</b>')
fig = clean_fig(fig)
write_image(fig, 'comparison_session_mean_time')
fig.show()

In [54]:
anomaly_df = df.copy()

dates, orig, lower, upper = [], [], [], []

for d in check_dates:
    subset_df = anomaly_df[anomaly_df.date <= d].tail(15).copy()
    values = subset_df.mean_seconds.values
    dates.append(subset_df.date.tolist()[-1])
    orig.append(values[-1])
    e = ema(values)
    s = std(values)
    lower.append(e - s * 2)
    upper.append(e + s * 2)

fig = go.Figure([
    go.Scatter(x=dates, y=lower, mode='lines', marker={'color': black}, name='upper', showlegend=False),
    go.Scatter(
        x=dates, y=upper, mode='lines', marker={'color': black}, name='upper', 
        fill='tonexty', fillcolor='rgba(240,236,225,20)', showlegend=False
    ),
    go.Scatter(
        x=dates, y=orig, name='Mean Seconds', line={'width': 4, 'color': dgreen},
        text=df[df.date.isin(dates)].weekday
        ),
])
fig.update_layout(title='<b>Anomaly detection using mean seconds per user</b>')
anomaly_date = '2022-03-21'
fig.add_annotation(
    x=anomaly_date, y=anomaly_df.loc[anomaly_df.date == anomaly_date, 'mean_seconds'].values[0],
    text='<b>Anomaly!</b>', ax=-100, ay=-20,
    showarrow=True, arrowhead=2, arrowsize=2, arrowcolor=black, 
    font={'family': 'Times New Roman', 'size': 20, 'color': orange}
)
fig = clean_fig(fig)
write_image(fig, 'anomaly_using_mean_time')
fig.show()

In [17]:
dates, orig, lower, upper = [], [], [], []


for d in check_dates:
    subset_df = (
        anomaly_df[anomaly_df.date <= d]
        .tail(45)
        .copy()
        .rename(columns={'date': 'ds', 'nbr_sessions': 'y'})
    )
    m = Prophet(daily_seasonality=False, yearly_seasonality=False)
    m.fit(subset_df[:-1])
    pred = m.predict(subset_df[-1:][['ds']])
    dates.append(d)
    orig.append(subset_df[-1:].y.values[0])
    lower.append(pred.yhat_lower.values[0])
    upper.append(pred.yhat_upper.values[0])

NameError: name 'Prophet' is not defined

In [ ]:
pred[['yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
fig = go.Figure([
    go.Scatter(x=dates, y=lower, mode='lines', marker={'color': black}, name='upper', showlegend=False),
    go.Scatter(x=dates, y=upper, mode='lines', marker={'color': black}, name='upper', fill='tonexty', fillcolor='rgba(240,236,225,20)', showlegend=False),
    go.Scatter(
        x=dates, y=orig, name='Nbr Sessions', line={'width': 4, 'color': dgreen},
        text=df[df.date.isin(dates)].weekday
        ),
])
fig = clean_fig(fig)
fig.show()

In [ ]:
forecast_df = df.rename(columns={'date': 'ds', 'nbr_sessions': 'y'}).copy()
m = Prophet()
m.fit(forecast_df[:-15])
pred_df = m.predict(forecast_df[-15:])

vis_df = forecast_df[-30:].copy()

fig = go.Figure([
    go.Scatter(x=vis_df.ds, y=vis_df.y, mode='lines', line={'color': dgreen, 'width': 3}, name='True Values'),
    go.Scatter(x=pred_df.ds, y=pred_df.yhat, mode='lines', line={'color': orange, 'width': 3, 'dash': 'dash'}, name='Prophet Forecast')
])
fig = clean_fig(fig)
fig.show()